This notebook attempt to fill NaN on past targets with the last values per farm.
This would allow the model to run even without the full past week's hourly targets, albeit giving suboptimal predictions.

In [ ]:
from sqlalchemy.engine import Engine, Connection
from sqlalchemy import create_engine
from os import getenv
from typing import List, Dict, Optional
import pandas as pd
from pandas import DataFrame, Series, Timestamp
import matplotlib.pyplot as plt
from datetime import datetime
import datetime as dt
import numpy as np
from numpy import ndarray
import pytz

In [ ]:
import sys
sys.path.insert(0, '..')

In [ ]:
from etl.ETL import db_connection, group_hourly

In [ ]:
try:
    from dotenv import load_dotenv

    load_dotenv()
except:
    print('No ".env" file or python-dotenv not installed... Using default env variables...')

In [ ]:
def db_connection() -> Engine:
    dbname: Optional[str] = getenv('POSTGRES_DB_NAME')
    host: Optional[str] = getenv('POSTGRES_HOST')
    user: Optional[str] = getenv('POSTGRES_USERNAME')
    password: Optional[str] = getenv('POSTGRES_PASSWORD')
    port: Optional[str] = getenv('POSTGRES_PORT')
        
    postgres_str: str = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'
    
    engine: Engine = create_engine(postgres_str)
    
    return engine

In [ ]:
startDate: str = (datetime.now(pytz.timezone('UTC')) - dt.timedelta(hours=167))
endDate: str = datetime.now(pytz.timezone('UTC'))
startDate, endDate 

In [ ]:
targetDate: datetime = endDate + dt.timedelta(hours=12)
targetDates: List[datetime] = [endDate + dt.timedelta(hours=i) for i in range(1,13)]

In [ ]:
start: str = startDate.strftime("%Y-%m-%d %H:00:00")
end: str = endDate.strftime("%Y-%m-%d %H:00:00")
targets: List[str] = [date.strftime("%Y-%m-%d %H:00:00") for date in targetDates]

In [ ]:
start, end

In [ ]:
targets

In [ ]:
farm_list: List = ['UP_PRCLCDPLRM_1',
'UP_PRCLCDMZRD_1',
'UP_PRCLCDPRZZ_1',
'UP_PRCLCMINEO_1',
'UP_PEPIZZA_1',
'UP_MPNTLCSMBC_1',
'UP_MPNTLCDMRN_1']

In [ ]:
query_tar: str = "SELECT * FROM sorgenia_energy WHERE start_date_utc >= '{}' and end_date_utc <= '{}'"
past_targets: DataFrame = pd.read_sql_query(query_tar.format(start, end), con=db_connection())
past_targets: DataFrame = group_hourly(past_targets)
past_targets: DataFrame = past_targets[past_targets['plant_name_up'].isin(farm_list)]

In [ ]:
past_targets

In [ ]:
tar_maserio = past_targets[past_targets['plant_name_up']=='UP_MPNTLCDMRN_1']

In [ ]:
tar_maserio

In [6]:
for i in range(1, 13):
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
